# Projeto 1 - Ciência dos Dados

Nome: __João Lucas Brasileiro de Sousa Lopes__

Nome: __Láisa Camilly de Oliveira__

Nome: __Nathan Braga__

Atenção: Serão permitidos grupos de até 4 pessoas, mas com uma rubrica mais exigente. 

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\Lucas\OneDrive\projeto 1 cdados\Projeto-1


Carregando a base de dados com as mensagens dos seus arquivos:

___
## Classificador automático


Faça aqui uma descrição do seu assunto e o contexto referente aos rótulos cujas mensagens (ou reviews) deverão ser classificadas.

ESCREVA AQUI...

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

#### <p style = 'color: green;'> <strong> Limpeza dos dados do df Treino: </strong>
- Transformação para letras minusculas.
- Tirou os espaços.
- Tirou os emojis.
- Tiramos os conectivos que aparecem bastante em ambos.

In [3]:
# funçao para limpeza de caracteres
import unidecode

def limpeza(texto):
    caracteres = '[´"!-.:?;\$\n'']'
    limpar = re.compile(caracteres)
    texto_limpo = re.sub(limpar, '', texto)

    texto_limpo = unidecode.unidecode(texto_limpo)
    return texto_limpo

In [4]:
train = pd.read_csv('dados_treino_TRIO_joaolbsl.csv')
train.head(5)

test = pd.read_csv('dados_teste_TRIO_joaolbsl.csv')
test.head(5)

,Comment,Topic
0,[158 page paper for reference. ](https://smart...,Physics
1,Try coating chemist. Application specialist (b...,Chemistry
2,"That there is 2-(decahydroquinolin-4-yl)-N,N-d...",Chemistry
3,sourcey source: [http://www.sheldoncomics.com/...,Physics
4,"Maybe with a rough enough surface, not as much...",Physics


In [5]:
for i in range(0,train.shape[0]):
    train['Comment'][i] = limpeza(train['Comment'][i])
    train['Comment'][i] = train['Comment'][i].lower().strip()
train

,Comment,Topic
0,dude probably doesn't want to drop several hun...,Chemistry
1,you don't think dissolving aspirin is hard i'm...,Chemistry
2,wanted to add in some more details that werent...,Physics
3,just buy precious metal silver or gold only th...,Chemistry
4,that's completely untrue,Chemistry
...,...,...
3495,do you know chinese whispers,Physics
3496,nepetalactone from catnip,Chemistry
3497,at this point i can never tell if qed is the l...,Physics
3498,thats why few people does inorganic chemistry,Chemistry


In [6]:
train_fis = train.loc[train['Topic'] == 'Physics', 'Comment'] # dados treino de fisica
train_qui = train.loc[train['Topic'] == 'Chemistry', 'Comment'] # dados treino de quimica

In [7]:
# Todas as palavras do df de fisica.
palavras_fis = []

for frase in train_fis:
    frase = frase.replace('\\n', ' ')
    frase = frase.split()

    for word in frase:
        palavras_fis.append(word)

# Todas as palavras do df de quimica.
palavras_qui = []

for frase in train_qui:
    frase = frase.replace('\\n', ' ')
    frase = frase.split()

    for word in frase:
        palavras_qui.append(word)


In [8]:
#pd.Series(palavras_fis).value_counts().head(40)

In [9]:
#pd.Series(palavras_qui).value_counts().head(40)

In [10]:
# Como se fosse nosso livro de português:
todas_palavras = palavras_fis + palavras_qui

In [13]:
# Classificador:

def calcula_prob (frase, palavras_topico, total_palavras_topico, todas_palavras):
    
    prob = 1
    frase = frase.split()
    for palavra in frase:
        if palavra in todas_palavras:
            prob = prob * (palavras_topico.count(palavra) + 1) / (total_palavras_topico + len(todas_palavras))

    return prob

In [14]:
total_palavras_fis = len(palavras_fis)
total_palavras_qui= len(palavras_qui)

In [15]:
# Teste:

frase = "the physics of the universe"
prob_fis = calcula_prob(frase, palavras_fis, total_palavras_fis, todas_palavras)
prob_qui = calcula_prob(frase, palavras_qui, total_palavras_qui, todas_palavras)

# Vendo o valor:
print(f"probabilidade de ser fisica: {prob_fis}")
print(f"probabilidade de ser biologia: {prob_qui}")

probabilidade de ser fisica: 6.830019490911401e-13
probabilidade de ser biologia: 2.5488276519468887e-15


In [26]:
# Todas as frases do df Treino:
frases_train = train['Comment']

# Todas as frases do df Teste:
frases_test = test['Comment']

In [27]:
topico_esperado = []

for frase in frases_train:

    prob_fis = calcula_prob(frase, palavras_fis, total_palavras_fis, todas_palavras)
    prob_qui = calcula_prob(frase, palavras_qui, total_palavras_qui, todas_palavras)

    if prob_fis > prob_qui:
        topico_esperado.append("Physics")
    else:
        topico_esperado.append("Chemistry")

train['Boot'] = topico_esperado

tabela_frequencia_relativa_train = pd.crosstab(train['Topic'], train['Boot'], normalize = True, margins =True)

print(tabela_frequencia_relativa_train)

Boot       Chemistry   Physics       All
Topic                                   
Chemistry   0.517429  0.038286  0.555714
Physics     0.092000  0.352286  0.444286
All         0.609429  0.390571  1.000000


In [28]:
#verdadeiros positivos
verdadeiros_positivos = tabela_frequencia_relativa_train['Chemistry']['Chemistry'] / tabela_frequencia_relativa_train['All']['Chemistry']
verdadeiros_positivos

0.9311053984575836

In [29]:
#falsos positivos 
falsos_positivos = tabela_frequencia_relativa_train['Chemistry']['Physics']/tabela_frequencia_relativa_train['All']['Physics']
falsos_positivos

0.20707395498392284

In [30]:
# Verdadeiros Negativos

verdadeiro_negativo = 100 - falsos_positivos
verdadeiro_negativo

99.79292604501607

In [31]:
#acuracia treino
acuracia_treino = (tabela_frequencia_relativa_train['Chemistry']['Chemistry'] + tabela_frequencia_relativa_train['Physics']['Physics'])*100
acuracia_treino

86.97142857142858

In [32]:
topico_esperado = []

for frase in frases_test:

    prob_fis = calcula_prob(frase, palavras_fis, total_palavras_fis, todas_palavras)
    prob_qui = calcula_prob(frase, palavras_qui, total_palavras_qui, todas_palavras)

    if prob_fis > prob_qui:
        topico_esperado.append("Physics")
    else:
        topico_esperado.append("Chemistry")

test['Boot'] = topico_esperado

tabela_frequencia_relativa_test = pd.crosstab(test['Topic'], test['Boot'], normalize = True, margins = True)

print(tabela_frequencia_relativa_test)

Boot       Chemistry   Physics       All
Topic                                   
Chemistry      0.448  0.126667  0.574667
Physics        0.132  0.293333  0.425333
All            0.580  0.420000  1.000000


In [33]:
#verdadeiros positivos

verdadeiro_positivo = tabela_frequencia_relativa_test['Chemistry']['Chemistry'] / tabela_frequencia_relativa_test['All']['Chemistry']
verdadeiro_positivo

0.7795823665893272

In [34]:
#falsos positivos 
falsos_positivos = tabela_frequencia_relativa_test['Chemistry']['Physics']/tabela_frequencia_relativa_test['All']['Physics']
falsos_positivos

0.3103448275862069

In [35]:
# Verdadeiros Negativos

verdadeiro_negativo = 100 - falsos_positivos
verdadeiro_negativo

99.6896551724138

In [36]:
#acuracia teste
acuracia_test = (tabela_frequencia_relativa_test['Chemistry']['Chemistry'] + tabela_frequencia_relativa_test['Physics']['Physics'])*100
acuracia_test

74.13333333333334

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

#### <p style = 'color: green;'> <strong> Classificador: </strong>

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações das mensagens entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Leia atentamente a rubrica colocada no enunciado do Projeto 1 (última página). <br>
Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nas mensagens, mas tendem a melhorar na classificação das mensagens. Ex: stemming, lemmatization, stopwords.
* CONSIDEROU arquivo com três categorias na classificação das variáveis (OBRIGATÓRIO PARA QUARTETOS, sem contar como item avançado)
* CONSTRUIU o cálculo das probabilidades corretamente utilizando bigramas E apresentou referência sobre o método utilizado.
* EXPLICOU porquê não pode usar novas mensagens classificadas pelo próprio classificador como amostra de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários diferentes, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* REFLETE criticamente sobre os resultados obtidos, identificando limitações do modelo e sugerindo possíveis melhorias ou diferentes abordagens com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa).
* DOCUMENTOU bem o código, com explicações claras para cada etapa do processo, incluindo o raciocínio por trás das decisões de modelagem e das transformações de dados.


___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[Natural Language Processing (Part 17)-Laplacian Smoothing](https://medium.com/@Coursesteach/natural-language-processing-part-17-laplacian-smoothing-7d4be71d0ded) **Mais simples**